# Import Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)

In [ ]:
gg_data = pd.read_csv('data/googleplaystore.csv')
# new_data = pd.read_csv('data/new_app_data.csv')

In [ ]:
data = gg_data.copy()

In [ ]:
for col in data:
    print(col)
    print(data[col].value_counts(normalize=True))
    print('-------------------------------')

**Observations**
1. **App - there seems to be a few duplicates in app. have to deal with deleting those.**
2. **Category - need to look into '1.9'.**
3. **Rating - the rating is from 1-5, but there's a rating of 19. Need to delete that one.**
4. **Reviews - might drop the '0' reviews or filter the amount of reviews.**
5. **Size - the 'varies with device' need to be replace with the average size and the "M" needs to be taken out.**
6. **Installs - need to get rid of the '+' and maybe filter a min-max scale.**
7. **Change Type into a numeric column**
8. **Change last update into days.**

# Cleaning

## App Duplicates

In [ ]:
# let's check to make sure we have duplicates
data[data['App'] == 'ROBLOX']

In [ ]:
data[data['App'] == 'ESPN']

In [ ]:
data[data['App'] == 'Candy Crush Saga']

**Looking at some of the duplicates it seems that the only difference is the amount of reviews and/or the category name. For now I think I'll stick with the first value and drop the rest.**

In [ ]:
data = data.drop_duplicates(subset=['App'], keep='first')

## Category

In [ ]:
data[data['Category'] == '1.9']

Since this is one entry I can easily find the correct info and plug it in.
Category = Lifestyle
Rating = 1.9
Reviews = 19.0
Size = 3.0M
Installs = 1,000
Type = Free
Price = 0
Content Rating = Everyone

In [ ]:
data.at[10472,"Category"]="LIFESTYLE"
data.at[10472, "Rating"] = 1.9
data.at[10472, "Reviews"] = "19.0"
data.at[10472, "Size"] = "3.0M"
data.at[10472, "Installs"] = "1,000+"
data.at[10472, "Type"] = "Free"
data.at[10472,'Price']= 0
data.at[10472, "Content Rating"] = "Everyone"
data.at[10472, "Last Updated"] = "February 11, 2018"

In [ ]:
data[data['App'] == 'Life Made WI-Fi Touchscreen Photo Frame']

## Last Updated

In [ ]:
data['Last Updated'] = pd.to_datetime(data['Last Updated'], format="%B %d, %Y", errors='coerce')
data['lastupdate'] = (data['Last Updated'] -  data['Last Updated'].max()).dt.days

In [ ]:
# data['Date'] = pd.to_datetime(data['Last Updated'])
# release_month = data['Last Updated'].dt.month
# data['Month'] = release_month

In [ ]:
data = data.drop(['Last Updated'], axis =1)

## Rating

In [ ]:
# replacing the NAN values with the average rating
data['Rating'] = data['Rating'].fillna(data['Rating'].mean())

In [ ]:
data.info()

## Add new data

In [ ]:
new_data = pd.read_csv('data/new_app_data.csv')
new_data.info()

In [ ]:
app_data = new_data.copy()
app_data

In [ ]:
app_data = app_data.dropna()
app_data

In [ ]:
df = pd.concat([data, app_data])
df.info()

In [ ]:
df['App'].value_counts(normalize=True)

In [ ]:
df = df.drop_duplicates(subset=['App'], keep='first')

In [ ]:
df= df.drop(columns={'Price','Current Ver', 'Android Ver'}, axis=1)

In [ ]:
df

## Reviews

In [ ]:
df.shape

In [ ]:
df = df.dropna().reset_index(drop =True)

In [ ]:
df.shape

In [ ]:
df.Reviews.value_counts(normalize=True)

In [ ]:
df['Reviews'] = pd.to_numeric(df['Reviews'])

## Size

In [ ]:
df[df['Size'] == "Varies with device"]

In [ ]:
df['Size'].value_counts(normalize=True)

In [ ]:
# create a variable to hold a certain value
mask = df['Size'].str.startswith('V')

In [ ]:
#replace that value with zero
df.loc[mask, 'Size'] = 0

In [ ]:
# convert the K and M to thousands and millions using regex
# df.Size = (df.Size.replace(r'[kMG]+$', '', regex=True).astype(float) * \
#            df.Size.str.extract(r'[\d\.]+([kMG]+)', expand=False).fillna(1)
#            .replace(['k','M', 'G'], [2**10, 2**20, 2**30]).astype(int))

#df['A'] = (df['A'] / 100).round(2)
# df['Size'] = (df['Size'] / 2**20).round(3)

In [ ]:
df['Size'] = df['Size'].astype('str')
df['Size'] = df['Size'].apply(lambda x: x.strip('k'))
df['Size'] = df['Size'].apply(lambda x: x.strip('M'))
df['Size'] = df['Size'].apply(lambda x: x.strip('G'))

In [ ]:
df['Size'] = df['Size'].astype('float')

In [ ]:
#to retain some data replace zeros with the column's avearge
mean_size = df['Size'].mean()
df['Size'] = df.Size.mask(df.Size == 0.0,mean_size)

In [ ]:
df.head(50)

## Installs

In [ ]:
# remove the + sign from columns
df['Installs'] = df['Installs'].str.replace('+','').str.replace(',','')

In [ ]:
df['Installs'] = pd.to_numeric(df['Installs'])

## Free Apps

In [ ]:
df.info()

In [ ]:
df.Type.value_counts(normalize=True)

In [ ]:
df['is_free'] = df['Type'].map(lambda x: 1 if x == 'Free' else 0)

In [ ]:
# df['Price'] = df['Price'].apply(lambda x : str(x).strip('$')).astype(float)
# df['Price'] = df['Price'].round()

In [ ]:
# from sklearn import preprocessing
# def labelencode(col, df):
#     le = preprocessing.LabelEncoder()
#     df[col] = le.fit_transform(df[col])

In [ ]:
# le = preprocessing.LabelEncoder()
# df['Content Rating'] = le.fit_transform(df['Content Rating'])

# le = preprocessing.LabelEncoder()
# df['Genres'] = le.fit_transform(df['Genres'])

In [ ]:
main_df = df.copy()

In [ ]:
main_df.head()

In [ ]:
main_df = main_df.drop('Type', axis=1)

# Cleaned Data

In [ ]:
main_df.info()

In [ ]:
main_df.to_csv('data_cleaned.csv')